# Movie Data Analysis

### Business Problem 
Microsoft sees all the big companies creating original video content and they want to get in on the fun. They have decided to create a new movie studio, but they don’t know anything about creating movies. You are charged with exploring what types of films are currently doing the best at the box office. You must then translate those findings into actionable insights that the head of Microsoft's new movie studio can use to help decide what type of films to create.

## Business Understanding

In order to generate actionable insights for this stakeholder, we are inspecting several data sets containing film information.  


### Sources:  
-[Box Office Mojo](https://www.boxofficemojo.com/)  
-[IMDB](https://www.imdb.com/)  
-[Rotten Tomatoes](https://www.rottentomatoes.com/)  
-[The Movie DB](https://www.themoviedb.org/)  
-[The Numbers](https://www.the-numbers.com/)   

### Topics to address:  
- Which producers create the best films?  
    - Criteria for analysis:  
        - Production budget vs. gross income
        - Mean of film (review) ratings over time
        - Success of particular genres of films per producer
        
- Which film genres are most successful in theaters?  
    - Criteria for analysis:  
        - Gross income of particular genres of films  
        - The associated production costs of particular genres
        - Success of particular film (classification) ratings  
        
- Is there a season during which film releases fare better?  
    - Criteria for analysis:  
        - Frequency of films being released over quarters and months  
        - Gross income of films grouped over quarters and months  
        - Production costs for films being released during the most profitable months

### Possible measurements of success:  
- Net income: gross income minus production budget
- Worldwide gross income
- Viewership numbers
- Film ratings

### 

## Data Preparation

## Data Understanding

## Evaluation  

### Recommendations for Film Producers  
-  

### Recommendations for Genres of Films  
-

### Recommendations for Release Dates  
-


## Conclusion